In [20]:
import pandas as pd
import unidecode
import requests
from bs4 import BeautifulSoup

data = pd.read_csv("votegetters_withratios.csv")

#remove dots
dot_removed = data['Player'].apply(lambda row: row.replace('.', ''))

#remove apostrophes
apost_removed = dot_removed.apply(lambda row: row.replace('\'', ''))

#remove accents
acc_removed = apost_removed.apply(lambda row: unidecode.unidecode(row))

#separate first and last names
separated_names = acc_removed.apply(lambda row: row.lower().split())

#take first five letters of last names
lastnames = separated_names.apply(lambda row: row[len(row)-1])
lastnames = lastnames.apply(lambda row: row[:5])

#take first two letters of first names
firstnames = separated_names.apply(lambda row: row[0])
firstnames = firstnames.apply(lambda row: row[:2])

initials = lastnames.apply(lambda row: row[0])

urls = "https://www.basketball-reference.com/players/" +  initials + "/" + lastnames + firstnames + "01.html"

newdata = pd.DataFrame([], columns=['Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP'])
for i in range(len(urls)):
    same = 0
    playernum = 1
    while same == 0:
        print(urls[i])
        page = requests.get(urls[i])
        soup = BeautifulSoup(page.text, 'lxml')
        table = soup.find_all('table')
        pergame = pd.read_html(str(table))[0]
        try:
            advanced = pd.read_html(str(table))[4]
        except IndexError:
            advanced = pd.read_html(str(table))[2]
        years = pergame['Season'].apply(lambda row: str(row)[5:])
        adv_years = advanced['Season'].apply(lambda row: str(row)[5:])
        '''print(years)
        print(adv_years)'''
        year = str(data['Year'][i])[2:]
        print("data['Year'][i]:", data['Year'][i])
        if year in years.tolist():
            j = years.where(years == year).dropna().index[0] #index of that year
            print("year:", year, "j:", j)
            try:
                k = adv_years.where(adv_years == year).dropna().index[0]
                print("year:", year, "j:", j, "k:", k)
                year_advanced = pd.DataFrame(advanced.iloc[k]).T
            except IndexError:
                playernum += 1
                urls[i] = urls[i].replace(str(playernum-1), str(playernum))
                continue
            #marker
            year_pergame = pd.DataFrame(pergame.iloc[j]).T
            try:
                float(year_pergame.loc[j, 'PTS'])
            except ValueError:
                playernum += 1
                urls[i] = urls[i].replace(str(playernum-1), str(playernum))
                continue
            if abs(float(year_pergame.loc[j, 'PTS']) - data.loc[i, 'PTS']) < 0.01 and year_pergame.loc[j, 'Season'] == year_advanced.loc[k, 'Season']:
                same = 1
                playernum = 1
                year_advanced.index = year_pergame.index
                newrow = pd.concat([year_pergame, year_advanced], axis=1)
                print("year_pergame.shape:", year_pergame.shape)
                print("year_advanced.shape:", year_advanced.shape)
                print("newrow.shape:", newrow.shape)
                print(newrow)
                newdata = pd.concat([newdata, newrow], ignore_index="True")
            else:
                playernum += 1
                urls[i] = urls[i].replace(str(playernum-1), str(playernum))
            #marker
        else:
            playernum += 1
            urls[i] = urls[i].replace(str(playernum-1), str(playernum))

print(urls)
print(len(urls))
newdata

https://www.basketball-reference.com/players/a/abdulka01.html
data['Year'][i]: 1980
year: 80 j: 10
year: 80 j: 10 k: 10
year_pergame.shape: (1, 30)
year_advanced.shape: (1, 29)
newrow.shape: (1, 59)
     Season   Age   Tm   Lg Pos     G   GS    MP    FG   FGA  ... Unnamed: 19  \
10  1979-80  32.0  LAL  NBA   C  82.0  NaN  38.3  10.2  16.9  ...         NaN   

    OWS  DWS    WS  WS/48 Unnamed: 24 OBPM DBPM  BPM VORP  
10  9.5  5.3  14.8  0.227         NaN  4.8  2.4  7.2  7.3  

[1 rows x 59 columns]
https://www.basketball-reference.com/players/e/ervinju01.html
data['Year'][i]: 1980
year: 80 j: 8
year: 80 j: 8 k: 8
year_pergame.shape: (1, 30)
year_advanced.shape: (1, 29)
newrow.shape: (1, 59)
    Season   Age   Tm   Lg Pos     G    GS    MP    FG   FGA  ... Unnamed: 19  \
8  1979-80  29.0  PHI  NBA  SF  78.0  78.0  36.1  10.7  20.7  ...         NaN   

   OWS  DWS    WS  WS/48 Unnamed: 24 OBPM DBPM  BPM VORP  
8  7.3  5.2  12.5  0.213         NaN  5.7  1.9  7.6  6.8  

[1 rows x 59 colu

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP
0,1979-80,32.0,LAL,NBA,C,82.0,NaN,38.3,10.2,16.9,...,NaN,9.5,5.3,14.8,0.227,NaN,4.8,2.4,7.2,7.3
1,1979-80,29.0,PHI,NBA,SF,78.0,78.0,36.1,10.7,20.7,...,NaN,7.3,5.2,12.5,0.213,NaN,5.7,1.9,7.6,6.8
2,1979-80,27.0,SAS,NBA,SG,78,NaN,37.6,13.1,24.9,...,NaN,9.3,1.3,10.6,0.173,NaN,5.5,-1.6,3.9,4.4
3,1979-80,23.0,BOS,NBA,PF,82,82,36.0,8.5,17.8,...,NaN,5.6,5.6,11.2,0.182,NaN,3.0,1.5,4.5,4.8
4,1979-80,31.0,BOS,NBA,PG,80,80,35.8,4.8,9.9,...,NaN,5.9,2.9,8.9,0.148,NaN,1.4,-0.3,1.1,2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,2020-21,32.0,WAS,NBA,PG,65.0,65.0,36.4,8.4,19.0,...,NaN,0.5,3.2,3.7,0.075,NaN,2.6,1.1,3.7,3.4
681,2020-21,24.0,PHI,NBA,PG,58,58,32.4,5.6,10.1,...,NaN,2.7,3.3,6.0,0.153,NaN,0.7,2.2,2.9,2.3
682,2020-21,31.0,TOT,NBA,"PG,SG",44.0,43.0,36.6,7.8,16.7,...,NaN,5.3,1.7,7.0,0.208,NaN,6.2,1.0,7.2,3.7
683,2020-21,36.0,LAL,NBA,PG,45.0,45.0,33.4,9.4,18.3,...,NaN,3.0,2.6,5.6,0.179,NaN,5.9,2.3,8.1,3.8


In [24]:
newdata.to_csv("pergame_advanced.csv")